In [ ]:
import requests
import csv
import random
import time

API_KEY = "API_KEY"  # Deinen API-Schlüssel hier einfügen
TOTAL_BOOKS = 500
BATCH_SIZE = 40
TIMEOUT = 10  # Sekunden

keywords = ["a", "e", "the", "book", "story", "novel", "history", "science", "life", "adventure"]

books_data = []

while len(books_data) < TOTAL_BOOKS:
    term = random.choice(keywords)
    start_index = random.randint(0, 900)
    url = f"https://www.googleapis.com/books/v1/volumes?q={term}&startIndex={start_index}&maxResults={BATCH_SIZE}&key={API_KEY}"
    
    try:
        response = requests.get(url, timeout=TIMEOUT)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.Timeout:
        print("Request timed out. Warte kurz und versuche erneut...")
        time.sleep(2)
        continue
    except requests.exceptions.RequestException as e:
        print(f"Request-Fehler: {e}")
        time.sleep(2)
        continue

    if "items" not in data:
        continue
    
    for item in data["items"]:
        info = item.get("volumeInfo", {})
        sale_info = item.get("saleInfo", {})

        title = info.get("title", "")
        authors = ", ".join(info.get("authors", []))
        categories = ", ".join(info.get("categories", []))
        description = info.get("description", "")
        rating = info.get("averageRating", "")
        image = info.get("imageLinks", {}).get("thumbnail", "")
        
        list_price = ""
        if sale_info.get("listPrice"):
            list_price = f"{sale_info['listPrice'].get('amount')} {sale_info['listPrice'].get('currencyCode')}"
        
        books_data.append({
            "Title": title,
            "Authors": authors,
            "Genre": categories,
            "Price": list_price,
            "Rating": rating,
            "Description": description,
            "Image": image
        })
        
        if len(books_data) >= TOTAL_BOOKS:
            break
    
    time.sleep(0.3)  # kurze Pause, um Timeouts zu reduzieren

# CSV speichern
keys = ["Title", "Authors", "Genre", "Price", "Rating", "Description", "Image"]
with open("books_random5.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=keys)
    writer.writeheader()
    writer.writerows(books_data)

print(f"{len(books_data)} Bücher gespeichert in 'books_random.csv'")